# COVID-19 Data Analysis
**Joe Corliss**

[Data source](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

## Execution Options

In [ ]:
# Config
DATA_PATH = '../data/'
random_state = 0

# Modeling
metric = 'confirmed'  # Variable of interest
metric_min = {'confirmed': round(2**9), 'deaths': round(2**3)}  # Filter out samples where the metric is below this value
log_metric = True  # Take log of the metric?
days_history = 7  # Number of consecutive past days to use for prediction (including the current day)
days_horizon = 7  # Number of days ahead to predict

In [ ]:
# Validate input
assert isinstance(DATA_PATH, str), "Bad input for option 'DATA_PATH'"
assert isinstance(random_state, int), "Bad input for option 'random_state'"
assert metric in {'confirmed', 'deaths'}, "Bad input for option 'metric'"
assert isinstance(metric_min, dict), "Bad input for option 'metric_min'"
assert isinstance(log_metric, bool), "Bad input for option 'log_metric'"
assert isinstance(days_history, int) and days_history >= 1, "Bad input for option 'days_history'"
assert isinstance(days_horizon, int) and days_horizon >= 1, "Bad input for option 'days_horizon'"

## Imports

In [ ]:
import sys

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [ ]:
# Check package versions
assert sys.version[:3] == '3.6', 'Unexpected Python version: expected 3.6, got {}'.format(sys.version[:3])
assert mpl.__version__.rpartition('.')[0] == '3.1', 'Unexpected matplotlib version: expected 3.1, got {}'.format(mpl.__version__.rpartition('.')[0])
assert np.__version__.rpartition('.')[0] == '1.18', 'Unexpected numpy version: expected 1.18, got {}'.format(np.__version__.rpartition('.')[0])
assert pd.__version__.rpartition('.')[0] == '1.0', 'Unexpected pandas version: expected 1.0, got {}'.format(pd.__version__.rpartition('.')[0])
assert sklearn.__version__.rpartition('.')[0] == '0.22', 'Unexpected scikit-learn version: expected 0.22, got {}'.format(sklearn.__version__.rpartition('.')[0])

In [ ]:
from sklearn import linear_model
from sklearn import model_selection

In [ ]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

## Load data

In [ ]:
values = pd.read_csv(DATA_PATH + 'time_series_covid19_{metric}_global.txt'.format(metric=metric))

In [ ]:
print('Data updated to:', values.columns[-1])

In [ ]:
values.info(verbose=True)

Check that there is no unexpected missing data:

In [ ]:
assert values.notnull().all()[1:].all(), 'Unexpected missing data!'

In [ ]:
values.drop(columns={'Lat', 'Long'}, inplace=True)

In [ ]:
values.rename(columns={'Country/Region': 'country', 'Province/State': 'province'}, inplace=True)

In [ ]:
values.sample(5)

## Look at Countries Split Up by Province/State

Number of distinct provinces for countries with more than one province:

In [ ]:
country_counts = values['country'].value_counts()
print(country_counts[country_counts > 1])

List current value by country and province, sorted by country ascending, then value descending:

In [ ]:
values.loc[values['country'].map(country_counts > 1)].iloc[:, [1, 0, -1]].sort_values(
    ['country', values.columns[-1]], ascending=[True, False]
)

## Data Processing

Add a new variable `locale` that combines the country and provice (if applicable):

In [ ]:
values.insert(0, 'locale', values['country'].where(
    values['province'].isnull(),
    values['country'] + '/' + values['province']
))

Replaces the `country` and `province` columns:

In [ ]:
values.drop(columns={'province', 'country'}, inplace=True)

In [ ]:
values.sample(5)

Plot values for each locale (repeat the second cell below):

In [ ]:
idx = 0

In [ ]:
if idx < values.shape[0]:
    while values.iloc[idx, -1] < 100:
        idx += 1
    plt.figure(dpi=100)
    values.iloc[idx, 1:].plot.line(marker='.', markersize=5, logy=log_metric)
    plt.title('Locale #{idx}: {locale}'.format(idx=idx + 1, locale=values.iloc[idx, 0]))
    plt.xlabel('Date')
    plt.ylabel(metric.capitalize())
    plt.grid(True)
    idx += 1

Stack the data:

In [ ]:
values_melt = values.melt(id_vars=['locale'], var_name='date', value_name='value')

In [ ]:
values_melt['date'] = pd.to_datetime(values_melt['date'])

In [ ]:
values_melt.sample(5)

Filter by the minimum required metric value:

In [ ]:
values_melt = values_melt.loc[values_melt['value'] >= metric_min[metric]]

Apply log to the metric values, if specified:

In [ ]:
if log_metric:
    values_melt['value'] = values_melt['value'].map(np.log10)

In [ ]:
values_melt.sample(5)

## Construct Final Datasets

In [ ]:
final_schema = pd.DataFrame(columns=['locale', 'date']
                                    + ['value_d{}'.format(idx) for idx in range(-days_history + 1, 1)]
                                    + ['value_d+{}'.format(days_horizon)])

In [ ]:
final_schema

In [ ]:
train_to_append_list = []
pred_to_append_list = []

In [ ]:
for locale in sorted(values_melt['locale'].unique()):
    values_melt_locale = values_melt.loc[values_melt['locale'] == locale].sort_values('date')
    
    # Add to training data
    for idx in range(values_melt_locale.shape[0] - days_history - days_horizon + 1):
        train_to_append = values_melt_locale.iloc[idx + (days_history - 1), :2].to_dict()
        train_to_append.update(zip(final_schema.columns[2:-1], values_melt_locale['value'].iloc[idx : idx + days_history]))
        train_to_append[final_schema.columns[-1]] = values_melt_locale['value'].iloc[idx + (days_history - 1) + days_horizon]
        train_to_append_list.append(train_to_append)
        
    # Add to prediction data
    if days_history <= values_melt_locale.shape[0]:  # Can only predict if we have enough days of history
        pred_to_append = values_melt_locale.iloc[-1, :2].to_dict()
        pred_to_append.update(zip(final_schema.columns[2:-1], values_melt_locale['value'].iloc[-days_history:]))
        pred_to_append_list.append(pred_to_append)

In [ ]:
train = final_schema.append(train_to_append_list, ignore_index=True)
pred = final_schema.append(pred_to_append_list, ignore_index=True)

In [ ]:
train.set_index(['locale', 'date'], inplace=True, verify_integrity=True)
pred.set_index(['locale', 'date'], inplace=True, verify_integrity=True)

In [ ]:
train.head()

In [ ]:
pred.head()

Check the size of the dataset:

In [ ]:
print('The training data has {} samples'.format(train.shape[0]))
print('Predictions will be made for {} locales'.format(pred.shape[0]))

Shuffle the training data:

In [ ]:
train = train.sample(frac=1, random_state=random_state)

## Model Training and Validation

In [ ]:
model = linear_model.LassoCV(
    eps=1e-8,
    fit_intercept=True,
    max_iter=100000,
    n_jobs=-1,
    random_state=random_state,
)

In [ ]:
model.fit(train.iloc[:, :-1], train.iloc[:, -1])

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
model.alpha_

In [ ]:
min(model.alphas_), max(model.alphas_)

Cross-validated mean absolute error:

In [ ]:
mean_abs_errors = -model_selection.cross_val_score(
    estimator=linear_model.Lasso(
        alpha=model.alpha_,
        max_iter=100000,
        random_state=random_state,
    ),
    X=train.iloc[:, :-1],
    y=train.iloc[:, -1],
    scoring='neg_mean_absolute_error',
)

In [ ]:
sorted(mean_abs_errors)

## Model Selection Information

In [ ]:
pd.DataFrame(
    data={
        'days_history': days_history,
        'days_selected': (model.coef_ != 0).sum(),
        'metric_min': metric_min[metric],
        'n_samples': train.shape[0],
        'samples_per_param': train.shape[0]/(days_history + 1),
        'mean_error': mean_abs_errors.mean(),
    },
    index=[0],
)

## Predictions

In [ ]:
pred['value_d+{}'.format(days_horizon)] = model.predict(pred.iloc[:, :-1])

In [ ]:
if log_metric:
    for col in pred.columns:
        pred[col] = pred[col].map(lambda x: round(10**x))

In [ ]:
pred